In [1]:
import pandas as pd
import numpy as np
import iqplot

# Bokeh es la biblioteca que usaremos para las gráficas
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, HoverTool, NumeralTickFormatter, DataTable, TableColumn, NumberFormatter
from bokeh.palettes import Dark2, YlGn9, Viridis, Pastel1

# output_notebook() activa el despliegue de gráficas en un notebook.
output_notebook()

Loading BokehJS ...

In [42]:
df = pd.read_csv("answers-2022.csv", index_col=0)
# El salario de las personas de Mx se guarda en la columna salarymx y está en pesos.
# En la mayoría de los casos solo vamos a tomar en cuenta los datos de personas en Mx con salario en pesos.
df = df[(df['country'] == 'México')]
df.count()

created          1889
salarymx         1889
salaryusd        1889
extramx          1889
extrausd         1889
                 ... 
covid_salario      19
covid_carga        19
covid_apoyo      1397
vacaciones       1479
aguinaldo        1487
Length: 197, dtype: int64

In [43]:
df.salarymx.describe()

count      1889.000000
mean      54933.848597
std       39816.068573
min        5000.000000
25%       28000.000000
50%       46001.000000
75%       70000.000000
max      350000.000000
Name: salarymx, dtype: float64

In [44]:
df.experience.describe()

count    1889.000000
mean       10.543674
std         7.703615
min         0.000000
25%         5.000000
50%         9.000000
75%        15.000000
max        40.000000
Name: experience, dtype: float64

## Experiencia

In [45]:
df['exp_bin'] = pd.cut(df['experience'],bins=[-0.1,2,4,6,8,10,14,20,40])

exp = df.groupby('exp_bin')['salarymx'].agg(['count', 'median','mean', 'std'])
exp['std'] = round(exp['std']).astype(int)
# Groupby deja exp_bin como un índice, lo necesitamos como una columna normal así que damos reset_index.
exp = exp.reset_index()
exp.head(10)

,exp_bin,count,median,mean,std
0,"(-0.1, 2.0]",211,23000.0,26306.990521,15611
1,"(2.0, 4.0]",255,30000.0,36138.137255,21939
2,"(4.0, 6.0]",248,39950.0,45981.548387,31715
3,"(6.0, 8.0]",183,50000.0,56724.721311,36829
4,"(8.0, 10.0]",226,55194.0,62695.101770,42163
5,"(10.0, 14.0]",259,62000.0,66007.138996,32414
6,"(14.0, 20.0]",300,60000.0,66223.830000,40758
7,"(20.0, 40.0]",207,60000.0,77719.328502,57540


In [6]:

# Los valores en exp_bin son intervalos pero para la gráfica necesitamos que sean strings/categoricas.
exp_labels = ['0-2', '3-4', '5-6', '7-8', '9-10', '11-14', '15-20', '20+']
exp['exp_bin'] = exp_labels

# Tomo la paleta YlGn9 que tiene 9 colores y uso los primeros 8. No uso YlGn8 porque el último tono es muy claro.
# Pongo el resultado en una variable aux para luego poder invertir el orden de colores (más claro al principio)
aux = YlGn9[0:8]
exp['color'] = aux[::-1]

src = ColumnDataSource(exp)
p = figure(x_range=exp_labels, plot_height=400, plot_width=700)
p.vbar(source=src, x='exp_bin', top='median', width=0.95, color='color')
p.title.text = 'Salario medio de acuerdo a a la experiencia'
p.xaxis.axis_label = 'Experiencia (años)'
p.yaxis.axis_label = 'Salario bruto mensual (MXN)'
p.yaxis.formatter = NumeralTickFormatter(format='$0 a')

hover = HoverTool()
hover.tooltips=[
    ('Experiencia', '@exp_bin años'),
    ('Observaciones', '@count'),
    ('Salario medio', '@median{$0,0}'),
]
p.add_tools(hover)

show(p)

## Comparación por género

In [69]:
gender = df.groupby(['gender'])['salarymx'].agg(['count','median','mean', 'std']).fillna(0)
gender.head(25)

,count,median,mean,std
gender,,,,
hombre,1502,50000.0,58182.494008,41218.107946
mujer,372,35000.0,41843.908602,29768.814133
nb,15,38000.0,54266.666667,49722.396027


In [64]:
p = iqplot.box(
    data=df[df.profile.isin(['godin'])],
    q="salarymx",
    cats=["gender"],
    frame_width=800,
)

p.xaxis.formatter = NumeralTickFormatter(format='$0 a')
p.xaxis.axis_label = 'Salario bruto mensual (MXN) para profesionistas TI'


show(p)

In [7]:
# Agrupamos por experiencia y género. Llenamos con 0 los grupos sin valores.
gender = df.groupby(['exp_bin', 'gender'])['salarymx'].agg(['median','count']).fillna(0)
gender.head(25)

median  count
exp_bin      gender                 
(-0.1, 2.0]  hombre   20000.0    186
             mujer    18500.0     62
             nb       25500.0      5
(2.0, 4.0]   hombre   30000.0    201
             mujer    29500.0     72
             nb       23500.0      3
(4.0, 6.0]   hombre   35000.0    217
             mujer    35000.0     60
             nb       23500.0      2
(6.0, 8.0]   hombre   50000.0    154
             mujer    33000.0     53
             nb           0.0      0
(8.0, 10.0]  hombre   55000.0    210
             mujer    34000.0     45
             nb           0.0      0
(10.0, 14.0] hombre   60000.0    246
             mujer    40000.0     48
             nb       27500.0      4
(14.0, 20.0] hombre   55000.0    289
             mujer    45000.0     47
             nb       56000.0      1
(20.0, 40.0] hombre   60000.0    196
             mujer    40000.0     25
             nb      148000.0      2

In [8]:
# El dataframe que arroja el groupby no se presta a graficar, así que lo reacomodamos en uno nuevo.
# El número de observaciones de género no binario es muy bajo y no arroja datos robustos así que lo omitiré.
data = {
    'exp': exp_labels,
    'hombre_salary' : list(gender.xs('hombre',level=1)['median']),
    'hombre_count' : list(gender.xs('hombre',level=1)['count']),
    'mujer_salary' : list(gender.xs('mujer',level=1)['median']),
    'mujer_count' : list(gender.xs('mujer',level=1)['count'])
}
genderdf = pd.DataFrame(data)
genderdf.head(10)

,exp,hombre_salary,hombre_count,mujer_salary,mujer_count
0,0-2,20000.0,186,18500.0,62
1,3-4,30000.0,201,29500.0,72
2,5-6,35000.0,217,35000.0,60
3,7-8,50000.0,154,33000.0,53
4,9-10,55000.0,210,34000.0,45
5,11-14,60000.0,246,40000.0,48
6,15-20,55000.0,289,45000.0,47
7,20+,60000.0,196,40000.0,25


In [9]:
# Teniendo esta estructura más amigable procedamos a generar lineas con sus tooltips.
src = ColumnDataSource(genderdf)
p = figure(x_range=exp_labels, plot_height=400)

renderer = p.line(x='exp',y='hombre_salary', source=src, color ='#1f77b4', line_width=2, legend_label='hombre')
p.add_tools(HoverTool(
            renderers=[renderer],
            tooltips=[
                ('Género', 'Hombre'),
                ('Experiencia', '@exp años'),
                ('Observaciones', '@hombre_count'),
                ('Salario', '@hombre_salary{$0,0}')
            ]
        ))

renderer = p.line(x='exp',y='mujer_salary', source=src, color ='#e617e6', line_width=2, legend_label='mujer')
p.add_tools(HoverTool(
            renderers=[renderer],
            tooltips=[
                ('Género', 'Mujer'),
                ('Experiencia', '@exp años'),
                ('Observaciones', '@mujer_count'),
                ('Salario', '@mujer_salary{$0,0}')
            ]
        ))


p.title.text = 'Comparativo por género'
p.xaxis.axis_label = 'Experiencia (años)'
p.yaxis.axis_label = 'Salario bruto mensual (MXN)'
p.yaxis.formatter = NumeralTickFormatter(format='$0 a')
p.legend.location = "bottom_right"

show(p)


In [10]:

gender = df[(df['experience'] > 0)].groupby(["experience", "gender"])['salarymx'].agg(['median','count'])
gender = gender.reset_index()
gender = gender[(gender['count'] > 4)]
gender['size'] = round(np.sqrt(gender['count']))*2

# Agregamos una columna de color en base al valor de gender.
gender['color'] = ['#1f77b4' if x =='hombre' else '#e617e6' for x in gender['gender']] 

src = ColumnDataSource(gender)
p = figure(plot_height = 400, plot_width = 600, sizing_mode="scale_both", toolbar_location=None)
p.toolbar.active_drag = None
p.toolbar.active_scroll = None
p.circle(x='experience', y='median', source=src, size='size', color='color')
p.title.text = 'Comparativo por género'
p.xaxis.axis_label = 'Experiencia (años)'
p.yaxis.axis_label = 'Salario bruto mensual (MXN)'
p.yaxis.formatter = NumeralTickFormatter(format='$0 a')

hover = HoverTool()
hover.tooltips=[
        ('Género', '@gender'),    
        ('Experiencia', '@experience años'),
        ('Observaciones', '@count'),
        ('Salario medio', '@median{$0,0}'),
    ]

p.add_tools(hover)
show(p)

In [71]:
gx = pd.crosstab(index=df['city'], columns=df['gender'])
gx = gx[(gx['hombre'])>9]
gx.to_csv("tablas/gx.csv")
gx

gender,hombre,mujer,nb
city,,,
Aguascalientes,41,5,0
Cancún,15,2,0
Chihuahua,21,5,0
Colima,14,5,1
Cuernavaca,12,4,0
Culiacán,19,4,0
Estado de México,13,4,0
Guadalajara,193,48,0
Hermosillo,32,8,0


## Inglés

In [12]:
ingles = df.groupby("english_label")["salarymx"].agg(['count', 'median', 'mean', 'std']).sort_values(by=['median'], ascending=False)
ingles.head(20)

,count,median,mean,std
english_label,,,,
Nativo o bilingue (ILR 5),91,60000.0,71014.329670,60862.655137
Avanzado: Puedo conversar y escribir sin problemas sobre cualquier tema (ILR 4),537,58333.0,62179.027933,54135.753966
Profesional: Puedo interactuar profesionalmente con colegas y clientes (ILR 3),687,47000.0,51467.556041,36337.336933
Limitado: Me doy a entender pero con errores de gramática (ILR 2),567,32000.0,37769.890653,26772.627741
Elemental: Sé lo básico para sobrevivir (ILR 1),228,26000.0,29470.872807,21114.137538
Ninguno (ILR 0),18,18000.0,23583.333333,16930.315657


In [13]:
ingles = df.groupby(['english_num','exp_bin'])['salarymx'].agg(['median', 'count']).fillna(0)
data = {
    'exp': exp_labels,
    'Elemental': list(ingles.xs(1)['median']),
    'Limitado' : list(ingles.xs(2)['median']),
    'Profesional' : list(ingles.xs(3)['median']),
    'Avanzado' : list(ingles.xs(4)['median']),
    'Nativo' : list(ingles.xs(5)['median'])    
}

inglesdf = pd.DataFrame(data)

src = ColumnDataSource(inglesdf)

p = figure(x_range=exp_labels, plot_height=400, plot_width=600)

for col_name, color in zip(list(inglesdf.columns), Dark2[6]):
    if col_name == 'exp':
        continue

    p.add_tools(HoverTool(
        renderers= [p.line('exp', col_name, source=src, color=color, legend_label=col_name, line_width=2)],
        tooltips=[
            ('Ingles', col_name),
            ('Salario', '@'+col_name+'{$0,0}')
        ]
    ))

p.title.text = 'Salario de acuerdo al nivel de inglés'
p.xaxis.axis_label = 'Experiencia (años)'
p.yaxis.axis_label = 'Salario'
p.yaxis.formatter = NumeralTickFormatter(format='$0 a')
p.legend.location = "top_left"

    
show(p)


## Agrupación por ciudad y país

In [14]:
# Solo tomamos en cuenta a las personas con perfil de empleado (no freelancers, directivos ni emprendedores)
# Incluimos la experiencia como variable observada para obtener el promedio de experiencia por ciudad
cities = df[(df["profile"] == "godin")].groupby("city")['salarymx', 'experience'].agg(['count', 'median', 'mean', 'std'])
cities = cities.reset_index()

# El group by de múltiples columnas observadas con múltiples funciones agregadas nos genera que los nombre son tuplas
# así que hacemos este map para renombrar las columnas.
cities.columns = cities.columns.map('_'.join)

cities = cities[(cities["salarymx_count"]> 10)]
cities = cities[(cities["salarymx_median"]> 0)]
cities = cities.sort_values(by=['salarymx_median'], ascending=False)
src = ColumnDataSource(cities)
columns = [
        TableColumn(field="city_", title="Ciudad"),
        TableColumn(field="salarymx_count", title="n"),
        TableColumn(field="salarymx_median", title="Mediana", formatter=NumberFormatter(format='$0,0')),
        TableColumn(field="salarymx_mean", title="Media", formatter=NumberFormatter(format='$0,0')),
        TableColumn(field="salarymx_std", title="Des std", formatter=NumberFormatter(format='$0,0')),
        TableColumn(field="experience_mean", title="Experiencia promedio", formatter=NumberFormatter(format='0')),
    ]
table = DataTable(source=src,columns=columns,index_position=None, width=400)
show(table)

/var/folders/mt/4vyh28v93sn_p_vdvr053kxc0000gn/T/ipykernel_90825/2327758837.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  cities = df[(df["profile"] == "godin")].groupby("city")['salarymx', 'experience'].agg(['count', 'median', 'mean', 'std'])


In [15]:
#guardamos en un csv para importar a sheets
cities.to_csv("tablas/cities.csv")

In [40]:

top_cities = df[(df["profile"] == "godin") & (df["city"].isin(["Hermosillo", "Monterrey", "Querétaro", "Guadalajara", "Valle de México"]))]
top_cities.head()
p = iqplot.stripbox(data=top_cities, q="salarymx", cats="city", title="strip-box",jitter=True,
    marker_kwargs=dict(alpha=0.5), frame_width=800)
p.sizing_mode="stretch_width"
p.xaxis.axis_label = "Salario mensual bruto (MXN)"

p.xaxis.formatter = NumeralTickFormatter(format='$0 a')
show(p)

Hagamos el breakdown por país. Para ello nos basamos en la columna salaryusd.

In [41]:
# Volvemos a leer del csv para incluir datos de otros países
df2 = pd.read_csv("answers-2022.csv", index_col=0)
df2['salaryusd'] = np.where((df2.salaryusd == 0),round(df2.salarymx/19,0).astype(int), df2.salaryusd)
countries = df2.groupby("country")["salaryusd"].agg(['count', 'median', 'mean', 'std'])
countries = countries.reset_index()
countries = countries.sort_values(by=['median'], ascending=False)
countries.head(30)

,country,count,median,mean,std
7,Estados Unidos,113,7500.0,8490.893805,5674.618331
1,Canada,10,6250.0,6537.200000,2629.260386
4,Costa Rica,2,4275.0,4275.000000,2015.254326
9,México,1889,2421.0,2892.699841,2098.160685
6,España,11,2400.0,3746.454545,3895.680746
8,Guatemala,5,2300.0,2000.000000,1115.795680
10,Otro,15,2100.0,3244.866667,3467.531375
2,Chile,9,2000.0,2096.888889,1492.619212
5,Ecuador,8,1800.0,2615.250000,2336.834963
11,Perú,10,1437.5,1544.700000,1286.350311


## Lenguajes

El dilema con los lenguajes es que cada participante puede escoger varios lenguajes (máximo 3) que utiliza. Así que no podemos tener una única variable categórica para lenguaje, sino que tenemos una variable booleana (Y/N) por cada una de las opciones de lenguaje. Si tuvieramos una única variable, simplemente podríamos hacer un groupby y listo, pero al no tenerla, tenemos que "armar" nuestro dataframe.

Vamos a generar una lista donde cada elemento es a su vez una lista con la info de cada lenguaje (nombre del lenguaje, número de observaciones y salario medio). A partir de esa lista de listas generamos un dataframe y continuamos como de costumbre.

In [18]:
# Inicializamos nuestra lista maestra
lang_list = []
# Leemos la lista de lenguajes a partir de un catálogo.
import csv
with open("lang_options.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        lang_key = "lang_"+row['key']
        # Creamos una lista con el nombre del lenguaje, su num. de observaciones y salario medio, y agregamos dicha lista a nuestra lista maestra.
        lang_list.append([row['name'], df[(df[lang_key]=="Y")]["salarymx"].count(), df[(df[lang_key]=="Y")]["salarymx"].median(), df[(df[lang_key]=="Y")]["salarymx"].mean(), df[(df[lang_key]=="Y")]["salarymx"].std(), df[(df[lang_key]=="Y")]["experience"].mean()])

# Una vez que tenemos la lista maestra completa, creamos un dataframe indicando el nombre de las columnas.
langdf = pd.DataFrame(lang_list, columns = ['lenguaje', 'n', 'mediana','media', 'std', 'exp'])
# Agregamos una columna de popularidad que se calcule en base a un valor amortiguado del número de observaciones.        
langdf['popularidad'] = round(np.sqrt(langdf['n']/2)*2)
langdf.sort_values(by=['mediana'], ascending=False).head(40)





,lenguaje,n,mediana,media,std,exp,popularidad
15,Ruby,72,60000.0,55672.916667,50020.926563,7.736111,12.0
7,Groovy,24,60000.0,65909.916667,45322.008994,12.333333,7.0
11,Perl,4,60000.0,59000.000000,6633.249581,12.500000,3.0
0,Bash,79,50000.0,54997.506329,53855.456098,10.658228,13.0
10,Kotlin,19,50000.0,53968.421053,31651.786901,8.368421,6.0
5,Elixir,25,50000.0,49900.000000,67530.548643,9.160000,7.0
6,Go,46,48500.0,49456.521739,42564.829781,8.565217,10.0
8,Java,320,47450.0,54951.534375,46924.738517,9.884375,25.0
3,COBOL,7,47300.0,44942.857143,8884.041337,11.857143,4.0
14,Python,194,39500.0,44166.798969,38227.356353,8.118557,20.0


In [19]:
# Esto exporta el dataframe a un csv
langdf.to_csv("tablas/langs.csv")


In [20]:
from bokeh.models import LabelSet
from bokeh.models import Range1d

src = ColumnDataSource(langdf)
p = figure()
# Ponemos los circulos invisibles pero con tamaño para que sirvan los tooltips en hover.
p.circle(source=src, y='popularidad', x='mediana', line_color=None, fill_color=None, size=20)
p.yaxis.axis_label = 'Popularidad'

# La escala de popularidad tiene unidades arbitrarias así que prefiero evitar que se despliegue.
p.yaxis.major_label_text_font_size = '0pt'

p.xaxis.axis_label = 'Salario bruto mensual (MXN)'
p.xaxis.formatter = NumeralTickFormatter(format='$0 a')    
p.x_range = Range1d(10000, 70000)

labels = LabelSet(source=src, x='mediana', y='popularidad', text='lenguaje', level='glyph',
              x_offset=-10, y_offset=-5, render_mode='canvas', text_font_size="9pt", text_color='#1f77b4')
p.add_layout(labels)

hover = HoverTool()
hover.tooltips=[
    ('Lenguaje', '@lenguaje'),
    ('Observaciones', '@count'),
    ('Salario medio', '@salario{$0,0}'),
]

p.add_tools(hover)

show(p)

## Front end

In [21]:
# Inicializamos nuestra lista maestra
item_list = []
# Leemos la lista de opciones a partir de un catálogo.
import csv
with open("front_options.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        item_key = "front_"+row['key']
        # Por cada elemento en la lista agregamos un renglon con el nombre de la categoria y estadisticas agregadas.
        item_list.append([row['name'], df[(df[item_key]=="Y")]["salarymx"].count(), df[(df[item_key]=="Y")]["salarymx"].median(), df[(df[item_key]=="Y")]["salarymx"].mean(), df[(df[item_key]=="Y")]["salarymx"].std(), df[(df[item_key]=="Y")]["experience"].mean()])

# Una vez que tenemos la lista maestra completa, creamos un dataframe indicando el nombre de las columnas.
itemdf = pd.DataFrame(item_list, columns = ['Tecnologia front', 'n', 'mediana','media', 'std', 'exp'])
# Agregamos una columna de popularidad que se calcule en base a un valor amortiguado del número de observaciones.        
itemdf.sort_values(by=['mediana'], ascending=False).head(30)

,Tecnologia front,n,mediana,media,std,exp
3,Next.js,73,45000.0,45456.726027,32200.742292,7.205479
7,Spring Boot,119,40000.0,45276.403361,34441.838060,9.058824
0,Angular,250,35000.0,39703.340000,29127.721219,8.372000
4,React,358,35000.0,42308.497207,37360.941912,6.851955
5,Ruby on Rails,71,35000.0,42141.169014,35744.198740,6.859155
6,Semantic UI,5,32000.0,36100.000000,29896.488088,12.400000
1,Django,51,31150.0,38571.333333,30162.954939,7.607843
9,Vue.js,135,25000.0,31450.940741,25713.654439,7.170370
2,Laravel,83,20000.0,24869.361446,18510.002548,7.626506
8,svelte,10,11500.0,20450.000000,25925.587961,10.900000


In [22]:
itemdf.to_csv("tablas/front.csv")

## Certificaciones

In [23]:
# Inicializamos nuestra lista maestra
item_list = []
# Leemos la lista de opciones a partir de un catálogo.
import csv
with open("cert_options.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        item_key = "cert_"+row['key']
        # Por cada elemento en la lista agregamos un renglon con el nombre de la categoria y estadisticas agregadas.
        item_list.append([row['name'], df[(df[item_key]=="Y")]["salarymx"].count(), df[(df[item_key]=="Y")]["salarymx"].median(), df[(df[item_key]=="Y")]["salarymx"].mean(), df[(df[item_key]=="Y")]["salarymx"].std(), df[(df[item_key]=="Y")]["experience"].mean()])

# Una vez que tenemos la lista maestra completa, creamos un dataframe indicando el nombre de las columnas.
itemdf = pd.DataFrame(item_list, columns = ['Certificacion', 'n', 'mediana','media', 'std', 'exp'])
# Agregamos una columna de popularidad que se calcule en base a un valor amortiguado del número de observaciones.        
itemdf.sort_values(by=['mediana'], ascending=False).head(30)

,Certificacion,n,mediana,media,std,exp
8,Enterprise Governance (CGEIT),4,140000.0,147500.000000,92150.239645,22.250000
7,"Enterprise Architect (SEI, IASA, Togaf, Zachman)",29,80000.0,105547.931034,66277.279418,19.724138
20,SAP (cualquier módulo),30,63000.0,75545.000000,72613.830199,15.633333
13,ITIL Practitioner,62,60000.0,69451.612903,44916.230744,18.532258
19,PMP,68,60000.0,64650.000000,46704.534920,20.176471
2,AWS Developer,37,56000.0,55671.648649,33368.986500,11.054054
3,AWS Solution Architect,69,55000.0,52913.043478,32336.943510,10.840580
25,Testing Advanced (CTAL),10,54500.0,85900.000000,97929.055954,12.400000
6,COBIT,16,54000.0,59830.625000,45127.201770,18.187500
16,Microsoft Associate,74,52500.0,60003.567568,38095.534592,15.027027


In [24]:
itemdf.to_csv("tablas/certs.csv")

## Infraestructura

In [25]:
# Inicializamos nuestra lista maestra
item_list = []
# Leemos la lista de opciones a partir de un catálogo.
import csv
with open("infra_options.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        item_key = "infra_"+row['key']
        # Por cada elemento en la lista agregamos un renglon con el nombre de la categoria y estadisticas agregadas.
        item_list.append([row['name'], df[(df[item_key]=="Y")]["salarymx"].count(), df[(df[item_key]=="Y")]["salarymx"].median(), df[(df[item_key]=="Y")]["salarymx"].mean(), df[(df[item_key]=="Y")]["salarymx"].std(), df[(df[item_key]=="Y")]["experience"].mean()])

# Una vez que tenemos la lista maestra completa, creamos un dataframe indicando el nombre de las columnas.
itemdf = pd.DataFrame(item_list, columns = ['Infra', 'n', 'mediana','media', 'std', 'exp'])
# Agregamos una columna de popularidad que se calcule en base a un valor amortiguado del número de observaciones.        
itemdf.sort_values(by=['mediana'], ascending=False).head(30)

,Infra,n,mediana,media,std,exp
2,Consul,2,96000.0,96000.000000,50911.688245,5.500000
13,Spinnaker,4,91000.0,84750.000000,27705.294320,8.750000
1,Chef,2,75500.0,75500.000000,6363.961031,11.500000
5,Dynatrace,6,60500.0,57583.333333,25389.794538,9.833333
10,OpenShift,8,60000.0,69125.000000,20209.174013,9.375000
6,Istio,8,57500.0,55825.000000,33631.012474,12.375000
8,Kubernetes,75,55000.0,59772.000000,50571.310529,10.880000
7,Jenkins,52,54000.0,55007.692308,29881.464162,9.980769
14,Terraform,58,53000.0,51949.137931,38635.478754,9.689655
4,Docker,106,51000.0,50520.820755,34520.755937,10.056604


## Actividad

In [26]:
# Inicializamos nuestra lista maestra
master_list = []
# Leemos la lista de lenguajes a partir de un catálogo.
import csv
with open("activity_options.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        key = "act_"+row['key']
        master_list.append([row['name'], df[(df[key]=="Y")]["salarymx"].count(), df[(df[key]=="Y")]["salarymx"].median(), df[(df[key]=="Y")]["salarymx"].mean(), df[(df[key]=="Y")]["salarymx"].std()])

# Una vez que tenemos la lista maestra completa, creamos un dataframe indicando el nombre de las columnas.
act_df = pd.DataFrame(master_list, columns = ['actividad', 'n', 'mediana', 'mean', 'std'])
act_df.sort_values(by=['mediana'], ascending=False).head(30)

,actividad,n,mediana,mean,std
12,Dirección / Estrategia,183,70000.0,82120.437158,61740.331722
20,Preventa / Tech sales,45,70000.0,71782.222222,55264.489210
6,Coaching y mejora de procesos,105,60000.0,61764.666667,45835.087858
2,Arquitectura y diseño de sistemas,502,56125.0,61357.322709,47011.030168
25,Venta y desarrollo de negocios,41,55000.0,68673.170732,56593.988304
7,Consultoría de negocio,86,54000.0,63771.941860,46656.970523
18,Project management / Coordinación,288,53000.0,59013.565972,40530.573519
15,"Gestión de infraestructura (SysOps, DevOps)",177,47500.0,51399.474576,40911.560331
11,Dev relations,18,47000.0,43777.777778,39734.124548
21,Seguridad de información,76,46500.0,47838.592105,36757.448388


In [27]:
act_df.to_csv("tablas/actividad.csv")

## Estudios

In [28]:
educacion = df.groupby("education")["salarymx"].agg(['count', 'median', 'mean', 'std']).sort_values(by=['median'], ascending=False)
educacion['mean'] = round(educacion['mean']).astype(int)
educacion

,count,median,mean,std
education,,,,
maestria,341,52000.0,58715,44601.124020
phd,33,49000.0,56882,38588.513953
universidad,1075,40000.0,47513,39145.512505
posgrado,65,38000.0,56980,62064.423158
pasante,519,35000.0,45540,39693.465771
prepa,45,30000.0,42972,42619.301091
tecnica,45,20000.0,32576,34394.151770
secundaria,5,0.0,11200,16887.865466


## ¿Dónde aprendiste a programar?

In [29]:
edutype = df.groupby("edutype")["salarymx"].agg(['count', 'median', 'mean', 'std']).sort_values(by=['median'], ascending=False)
edutype

,count,median,mean,std
edutype,,,,
autodidacta,297,47000.0,54501.973064,48160.222688
escuela,1327,45000.0,51691.677468,41771.173177
trabajo,132,36500.0,41575.075758,26090.053472
online,312,30000.0,38338.108974,36884.447646
bootcamp,60,22000.0,25644.966667,21350.024753


Vemos que cursos online y bootcamps aparecen con los salarios más bajos. Pero eso puede ser engañoso, porque son opciones relativamente recientes y por lo tanto la gente que aprendió de esta manera no tiene tanta experiencia (que junto con el nivel de inglés es el factor que más influye en el salario). Así que ahora tomemos en cuenta solamente a los que tienen 5 años o menos de experiencia.

In [30]:
edutype = df[(df['experience']<=5)].groupby("edutype")["salarymx"].agg(['count', 'median', 'mean', 'std']).sort_values(by=['median'], ascending=False)
edutype

,count,median,mean,std
edutype,,,,
trabajo,27,35000.0,38270.518519,21266.209156
escuela,376,27000.0,30420.140957,21115.329268
autodidacta,57,26500.0,32509.666667,25198.594836
online,170,24000.0,27427.317647,23611.516302
bootcamp,49,20000.0,22548.938776,19944.000468


In [31]:
edutype.to_csv("tablas/edutype_5.csv")

Podemos ver que las cosas se emparejan mucho más. Incluso podemos decir que donde aprendiste a programar no es un factor significativo para tu salario. En otras palbras, lo que importa es lo que sabes, no como lo aprendiste.

Aprovechando que estamos en esto, vamos a ver si está cambiando la donde aprenden a programar las personas. Para ello, haremos un cruce de donde aprendieron vs años de experiencia.

In [32]:
exp_bin = df.groupby(['edutype','exp_bin'])['salarymx'].agg(['median', 'count']).fillna(0)
edutype_pct = exp_bin.groupby(level=1)['count'].apply(lambda x: x / float(x.sum()))
edutype_pct.head(20)

edutype      exp_bin     
autodidacta  (-0.1, 2.0]     0.075099
             (2.0, 4.0]      0.072464
             (4.0, 6.0]      0.125448
             (6.0, 8.0]      0.154589
             (8.0, 10.0]     0.117647
             (10.0, 14.0]    0.161074
             (14.0, 20.0]    0.192878
             (20.0, 40.0]    0.215247
bootcamp     (-0.1, 2.0]     0.118577
             (2.0, 4.0]      0.050725
             (4.0, 6.0]      0.017921
             (6.0, 8.0]      0.000000
             (8.0, 10.0]     0.031373
             (10.0, 14.0]    0.006711
             (14.0, 20.0]    0.002967
             (20.0, 40.0]    0.000000
escuela      (-0.1, 2.0]     0.505929
             (2.0, 4.0]      0.597826
             (4.0, 6.0]      0.591398
             (6.0, 8.0]      0.676329
Name: count, dtype: float64

In [33]:
data = {
    'exp': exp_labels,
    'escuela' : list(edutype_pct.xs('escuela')),
    'autodidacta': list(edutype_pct.xs('autodidacta')),
    'trabajo' : list(edutype_pct.xs('trabajo')),
    'online' : list(edutype_pct.xs('online')),
    'bootcamp' : list(edutype_pct.xs('bootcamp'))
}

edudf = pd.DataFrame(data)
edudf.sort_index(ascending=False, inplace=True)
edudf.head(20)

,exp,escuela,autodidacta,trabajo,online,bootcamp
7,20+,0.645740,0.215247,0.062780,0.076233,0.000000
6,15-20,0.679525,0.192878,0.065282,0.059347,0.002967
5,11-14,0.687919,0.161074,0.080537,0.063758,0.006711
4,9-10,0.592157,0.117647,0.086275,0.172549,0.031373
3,7-8,0.676329,0.154589,0.062802,0.106280,0.000000
2,5-6,0.591398,0.125448,0.068100,0.197133,0.017921
1,3-4,0.597826,0.072464,0.043478,0.235507,0.050725
0,0-2,0.505929,0.075099,0.023715,0.276680,0.118577


In [34]:


src = ColumnDataSource(edudf)
col_names = src.column_names

p = figure(x_range=exp_labels[::-1], plot_height=400, plot_width=600)

for col_name, color in zip(list(edudf.columns), Dark2[6]):
    if col_name == 'exp':
        continue

    p.add_tools(HoverTool(
        renderers= [p.line('exp', col_name, source=src, color=color, legend_label=col_name)],
        tooltips=[
            ('Formación', f'{col_name}'),
            ('Pct', '@'+col_name+'{0%}')
        ]
    ))

p.title.text = 'Evolución de donde aprendemos a programar'
p.xaxis.axis_label = 'Experiencia (años)'
p.yaxis.axis_label = 'Porcentaje'
p.yaxis.formatter = NumeralTickFormatter(format='0%')    
p.legend.location = (400,120)
    
show(p)


Llama la atención el crecimiento de los cursos online.

## Cliente local vs internacional

In [35]:
table = df.groupby("remote")["salarymx"].agg(['count', 'median', 'mean', 'std']).sort_values(by=['median'], ascending=False)
table.head(20)

,count,median,mean,std
remote,,,,
Y,1007,52500.0,55798.795432,43686.570822
N,1121,33290.0,42444.828724,38017.268036


In [36]:
table = df.groupby("orgtype")["salarymx"].agg(['count', 'median', 'mean', 'std']).sort_values(by=['median'], ascending=False)
table.head(20)

,count,median,mean,std
orgtype,,,,
startup,235,50000.0,57611.582979,53614.437391
itservices,932,45000.0,50583.859442,35662.130178
isv,362,40000.0,50505.074586,48810.061808
corp,414,38500.0,45991.946860,39222.032254
gobierno,87,24000.0,29279.482759,20531.093224
uni,79,24000.0,31200.797468,23756.368652
freelance,19,10000.0,39551.578947,80347.706132


## Covid-19

In [37]:
covid = df[["covid_carga","covid_remoto", "covid_salario", "covid_apoyo"]].copy()
covid.value_counts(["covid_remoto"])




covid_remoto  
remote            1745
semipresencial     203
onsite             180
dtype: int64

## Special cases

In [38]:
segment = df[(df["city"] == "Guadalajara") & (df["experience"]>6)]
ingles = segment.groupby("english_label")["salarymx"].agg(['count', 'median', 'mean', 'std']).sort_values(by=['median'], ascending=False)
ingles.head(20)


,count,median,mean,std
english_label,,,,
Avanzado: Puedo conversar y escribir sin problemas sobre cualquier tema (ILR 4),64,85500.0,85170.703125,44816.241333
Nativo o bilingue (ILR 5),13,80000.0,83538.461538,50172.395107
Profesional: Puedo interactuar profesionalmente con colegas y clientes (ILR 3),56,60000.0,65528.571429,26208.580928
Limitado: Me doy a entender pero con errores de gramática (ILR 2),21,40000.0,51714.285714,29809.315418
Elemental: Sé lo básico para sobrevivir (ILR 1),7,24000.0,33428.571429,27232.683445


In [39]:
ingles.to_csv("tablas/gdl_ingles.csv")